In [ ]:
import numpy as np
import os
import tensorflow as tf
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import KFold

###### Do not modify here ###### 

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

# training on MNIST but only on digits 0 to 4
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]

###### Do not modify here ###### 


In [ ]:
# Parameters setting
dim = 28 * 28
learning_rate = 0.01
num_steps = 100
batch_size = 2048
num_classes = 5 # 0-4
early_stop = 20
dropout = 0.5 

#################################### Build Execution Graph ###########################################
# Dataset definition
X = tf.placeholder(tf.float32, shape = (None, dim))
y = tf.placeholder(tf.int64, shape = (None))

# HW Hint
initializer = tf.contrib.layers.variance_scaling_initializer()
adam = tf.train.AdamOptimizer(learning_rate)
activation = tf.nn.elu

# Dropout
keep_prob = tf.placeholder(tf.float32)

# Build DNN model with five hidden layers of 128 neurons
hidden_layer = tf.layers.dense(X, 128, activation=activation, kernel_initializer=initializer)
hidden_layer = tf.nn.dropout(hidden_layer, keep_prob)
hidden_layer = tf.layers.dense(hidden_layer, 128, activation=activation, kernel_initializer=initializer)
hidden_layer = tf.nn.dropout(hidden_layer, keep_prob)
hidden_layer = tf.layers.dense(hidden_layer, 128, activation=activation, kernel_initializer=initializer)
hidden_layer = tf.nn.dropout(hidden_layer, keep_prob)
hidden_layer = tf.layers.dense(hidden_layer, 128, activation=activation, kernel_initializer=initializer)
hidden_layer = tf.nn.dropout(hidden_layer, keep_prob)
hidden_layer = tf.layers.dense(hidden_layer, 128, activation=activation, kernel_initializer=initializer)
# linear activation
out_layer = tf.layers.dense(hidden_layer, num_classes, kernel_initializer=initializer)

# Softmax
softY = tf.nn.softmax(out_layer)

# Compute loss
# tf.nn.sparse_softmax_cross_entropy_with_logits : combine softmax and cross_entropy
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=out_layer)
loss = tf.reduce_mean(loss)
optimizer = adam.minimize(loss)

# Compute accuracy
predlabelY = tf.nn.top_k(softY, 1).indices # label 0-4
predY = tf.nn.in_top_k(softY, y, 1) # boolean
accuracy = tf.reduce_mean(tf.cast(predY, tf.float32)) # boolean -> 0/1

init = tf.global_variables_initializer()
###########################################################################################################



In [ ]:
#################################### N-fold Cross Validaion ###########################################
N = 10
kf = KFold(n_splits=N)

saver = tf.train.Saver()

# Combine training and validation datasets
X_input = np.concatenate((X_train1, X_valid1), axis=0)
Y_input = np.concatenate((y_train1, y_valid1), axis=0)

print X_input.shape
print Y_input.shape

with tf.Session() as sess:
    sess.run(init)
    
    loss_list = []
    acc_list = []
    
    precision_set = np.zeros(num_classes)
    recall_set = np.zeros(num_classes)
    
    fold = 1
    for train_idx, val_idx in kf.split(X_input, Y_input):
        # Early-stop initialization
        best_loss = np.infty
        best_acc = 0
        patience = 0
        # Use id from kf(KFold) to get training and valudation data in each fold 
        trainX = X_input[train_idx]
        trainY = Y_input[train_idx]
        validationX = X_input[val_idx]
        validationY = Y_input[val_idx]
        
        num_batch = int(len(trainX) / batch_size) + 1
        
        for epoch in range(num_steps):
            # shuffle the data
            rd_idx = range(len(trainX))
            np.random.shuffle(rd_idx)

            # array_split : array , sections
            for idx in np.array_split(rd_idx, num_batch):
                batchX = trainX[idx]
                batchY = trainY[idx]
                sess.run(optimizer, feed_dict={ X: batchX, y: batchY, keep_prob: dropout })

            # Calculate the validation loss and accuracy
            val_loss, val_acc = sess.run([loss, accuracy], feed_dict={ X: validationX, y: validationY, keep_prob: 1.0 })
            estimatedY, estimatedlabelY = sess.run([predY, predlabelY], feed_dict={ X: validationX, y: validationY, keep_prob: 1.0 })
            
            # Calculate the precision and recall 
            precision = precision_score(validationY, estimatedlabelY, average=None)
            recall = recall_score(validationY, estimatedlabelY, average=None)
            
            # Check if loss is less than the best loss
            if val_loss < best_loss:
                best_loss = val_loss
                best_acc = val_acc
                patience = 0
                best_precision = precision
                best_recall = recall
                # Save the current network to a checkpoint file
                best_checkpoint = saver.save(sess, "./Team60_HW2_"+str(fold)+".ckpt")
            else:
                patience = patience + 1
                if patience > early_stop:
                    print "Early stop!"
                    break
                    
        loss_list.append(best_loss)
        acc_list.append(best_acc)
        precision_set = precision_set + best_precision
        recall_set = recall_set + best_recall
        print("Fold_N: {} Best_loss: {:.4f} Best_acc: {:.2f}".format(fold, best_loss, best_acc)) 
        fold = fold + 1
        
    # Average all n-fold models   
    print("Cross validation : loss {:.2f} accuracy {:.2f}".format(np.mean(loss_list), np.mean(acc_list))) 
    
    print "----------------------------"
    print "label precision recall"
  
    for i in range(num_classes):
        print("  {}     {:.2f}     {:.2f}".format(i, precision_set[i] / N, recall_set[i] / N))
    
    print "----------------------------"

In [ ]:
#################################### Training ###########################################
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

saver = tf.train.Saver()

###### Start TF session ######
with tf.Session() as sess:
    sess.run(init)

    # Early-stop initialization
    best_loss = np.infty
    best_acc = 0
    patience = 0
    
    num_batch = int(len(X_train1) / batch_size) + 1
    for epoch in range(num_steps):
            #shuffle
            rd_idx = range(len(X_train1))
            np.random.shuffle(rd_idx)

            # array_split : array , sections 
            for idx in np.array_split(rd_idx, num_batch):
                batchX = X_train1[idx]
                batchY = y_train1[idx]
                sess.run(optimizer, feed_dict={ X: batchX, y: batchY, keep_prob: dropout })

            val_loss, val_acc = sess.run([loss, accuracy], feed_dict={ X: X_valid1, y: y_valid1, keep_prob: 1.0 })
            estimatedY, estimatedlabelY = sess.run([predY, predlabelY], feed_dict={ X: X_valid1, y: y_valid1, keep_prob: 1.0 })
            
            precision = precision_score(y_valid1, estimatedlabelY, average=None)
            recall = recall_score(y_valid1, estimatedlabelY, average=None)
            
            print val_acc
            if val_loss < best_loss:
                best_loss = val_loss
                best_acc = val_acc
                best_precision = precision
                best_recall = recall
                patience = 0
                best_checkpoint = saver.save(sess, "./Team60_HW2.ckpt")
            else:
                patience = patience + 1
                if patience > early_stop:
                    print "Early stop!"
                    break
                    
    print("Best accuracy {:.2f}".format(best_acc*100)) 
    
    
    
    print "----------------------------"
    print "label precision recall"
  
    for i in range(num_classes):
        print("  {}     {:.2f}     {:.2f}".format(i, best_precision[i], best_recall[i]))
    
    print "----------------------------"
    

        

In [ ]:
#################################### Testing ###########################################
with tf.Session() as sess:
    
    saver.restore(sess, "./Team60_HW2.ckpt")

    val_loss, val_acc = sess.run([loss, accuracy], feed_dict={ X: X_test1, y: y_test1, keep_prob: 1.0 })
    estimatedY, estimatedlabelY = sess.run([predY, predlabelY], feed_dict={ X: X_test1, y: y_test1, keep_prob: 1.0 })
    print("Final test accuracy: {:.2f}%".format(val_acc * 100))
    
    
    precision = precision_score(y_test1, estimatedlabelY, average=None)
    recall = recall_score(y_test1, estimatedlabelY, average=None)
    
    print "----------------------------"
    print "label precision recall"
  
    for i in range(num_classes):
        print("  {}     {:.2f}     {:.2f}".format(i, precision[i], recall[i]))
    
    print "----------------------------"